In [10]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [11]:
my_link = pd.read_excel('../input_network/Nguyen_Dupuis_network.xlsx', sheet_name='link_info')
my_node = pd.read_excel('../input_network/Nguyen_Dupuis_network.xlsx', sheet_name='node_info')
my_link['name'] = my_link['from'].astype('str') +'-'+my_link['to'].astype('str')
my_demand = pd.read_excel('../input_network/Nguyen_Dupuis_network.xlsx', sheet_name='demand_info')

In [12]:
# generate a Graph
import networkx as nx
def get_graph(link_table, node_table):
    """
    generate a network based on link information and node information
    input: link_table and node table
    output: a graph

    Args:
        link_table ([pd.DataFrame]): [link table]
        node_table ([pd.DataFrame]): [node table]

    Returns:
        [graph]: [a graph for network]
    """

    # add eages from link table 
    G = nx.from_pandas_edgelist(df = link_table, 
                            edge_key= "link_id",
                            source = "from", 
                            target = "to", 
                            edge_attr=['name','link_id','type', 'attribute'], 
                            create_using=nx.MultiDiGraph())

    # add node attributes                       
    node_attributes = node_table.set_index('name').to_dict('index') # generate a node attributes dictionary 
    nx.set_node_attributes(G, node_attributes)

    return G

In [13]:
# generate a pathset
def get_path_set(G, origin_zone, destination_zone):
    """
    generate a path set and compute path attributes (O_node, D_node, links, cost, path flow, prob, logit_prob, y_flow) 

    Args:
        G ([graph]): [network]
        origin_zone ([list]): [list of origin zones]
        destination_zone ([list]): [list of destination zones]
        theta_1 ([float]): [parameter of logit model for path choice]
        
    Returns:
        [pd.DataFrame]: [path set]
    """

    # initialize a paths set
    paths = []
    for i in origin_zone:
        for j in destination_zone:
            for path in nx.all_simple_edge_paths(G, source= i, target= j): # generate all paths 
                od_pair = i +'-'+ j # record origin zone and destination zone
                links = path # record links
                link_sque =[G.edges[edge]['link_id'] for edge in path] # record links ID
                name_sque = [G.edges[edge]['name'] for edge in path]
                attribute_set = [G.edges[edge]['attribute'] for edge in path]            
                paths.append([od_pair, links, link_sque, name_sque, attribute_set])

    # generate a path information table based pandas DataFrame
    path_set = pd.DataFrame(paths, columns = ['OD_pair', 'links','link_sque','name_sque','attribute_set'])

    # map travel mode
    def map_mode(x):
        if x['attribute_set'] in [['board_auto', 'auto', 'alight_auto']]:
            return 'auto'
        elif x['attribute_set'] in [['board_transit', 'transit', 'alight_transit'],
                                    ['board_transit', 'transit', 'transit-transfer', 'alight_transit']]:
            return 'transit'
        elif x['attribute_set'] in [['board_auto', 'auto', 'P&R', 'transit', 'alight_transit']]:
            return 'P&R'
        elif x['attribute_set'] in [['board_ride-hailing', 'ride-hailing', 'R&T', 'transit', 'alight_transit'],
                                    ['board_ride-hailing', 'ride-hailing', 'R&T', 'transit', 'transit-transfer', 'alight_transit']]:
            return 'R&T'
        else:
            return 'other'

    path_set['attribute_set'] = path_set['attribute_set'].apply(lambda x: sorted(set(x),key=x.index))
    path_set['mode'] = path_set.apply(map_mode, axis=1)
    path_set = path_set[path_set['mode'].isin(['auto', 'P&R','transit', 'R&T'])]
    path_set['path_id'] = range(1, len(path_set)+1)
    path_set['path_id'] = 'path' + '-' + path_set['path_id'].astype('str')

    return path_set[['OD_pair','path_id','mode','link_sque','name_sque']]

In [14]:
my_graph = get_graph(link_table=my_link,node_table=my_node)
path_set = get_path_set(G=my_graph, origin_zone=['r1'], destination_zone=['s1'])
path_set

,OD_pair,path_id,mode,link_sque,name_sque
0,r1-s1,path-1,auto,"[43, 5, 2, 1, 3, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-..."
1,r1-s1,path-2,auto,"[43, 5, 2, 1, 3, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-..."
2,r1-s1,path-3,auto,"[43, 5, 2, 1, 3, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-..."
3,r1-s1,path-4,auto,"[43, 5, 2, 1, 3, 11, 15, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-..."
4,r1-s1,path-5,auto,"[43, 5, 2, 1, 3, 11, 15, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-..."
5,r1-s1,path-6,P&R,"[43, 5, 2, 1, 3, 38, 31, 32, 34, 47]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N22, N2..."
7,r1-s1,path-7,auto,"[43, 5, 2, 1, 4, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N7, N7-N11, N1..."
8,r1-s1,path-8,auto,"[43, 5, 6, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N7, N7-N11, N1..."
9,r1-s1,path-9,auto,"[43, 5, 6, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N11, ..."
10,r1-s1,path-10,auto,"[43, 5, 6, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N13, ..."


In [15]:
path_length =[]
path_travel= []
path_walk = []
path_wait = [] 
path_congestion = []
path_penalty = []
path_fare = []
for i in range(len(path_set)):
    link_sque = path_set['link_sque'].iloc[i]
    one_path = my_link[my_link['link_id'].isin(link_sque)]
    path_length.append(one_path['l_a'].sum())
    path_travel.append(one_path['travel'].sum())
    path_walk.append(one_path['walk'].sum())
    path_wait.append(one_path['wait'].sum())
    path_congestion.append(one_path['congestion'].sum())
    path_penalty.append(one_path['penalty'].sum())
    path_fare.append(one_path['fare_rate'].sum())

In [16]:
path_set['path_length'] = path_length
path_set['path_travel'] = path_travel
path_set['path_walk'] = path_walk
path_set['path_wait'] = path_wait
path_set['path_congestion'] = path_congestion
path_set['path_penalty'] = path_penalty
path_set['path_fare'] = path_fare

In [17]:
path_set

,OD_pair,path_id,mode,link_sque,name_sque,path_length,path_travel,path_walk,path_wait,path_congestion,path_penalty,path_fare
0,r1-s1,path-1,auto,"[43, 5, 2, 1, 3, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",21.0,0,0,0,0,0,0
1,r1-s1,path-2,auto,"[43, 5, 2, 1, 3, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",21.0,0,0,0,0,0,0
2,r1-s1,path-3,auto,"[43, 5, 2, 1, 3, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",26.0,0,0,0,0,0,0
3,r1-s1,path-4,auto,"[43, 5, 2, 1, 3, 11, 15, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-...",21.0,0,0,0,0,0,0
4,r1-s1,path-5,auto,"[43, 5, 2, 1, 3, 11, 15, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-...",26.0,0,0,0,0,0,0
5,r1-s1,path-6,P&R,"[43, 5, 2, 1, 3, 38, 31, 32, 34, 47]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N22, N2...",22.0,0,5,5,0,12,20
7,r1-s1,path-7,auto,"[43, 5, 2, 1, 4, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N7, N7-N11, N1...",19.0,0,0,0,0,0,0
8,r1-s1,path-8,auto,"[43, 5, 6, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N7, N7-N11, N1...",15.0,0,0,0,0,0,0
9,r1-s1,path-9,auto,"[43, 5, 6, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N11, ...",15.0,0,0,0,0,0,0
10,r1-s1,path-10,auto,"[43, 5, 6, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N13, ...",20.0,0,0,0,0,0,0


In [18]:
def get_path_info(path_set, link_table):
    """according to link-path matrix index to compute each path information

    Args:
        path_set ([pd.DataFrame]): [path set]

    Returns:
        [pd.DataFrame]: [path set after computing information]
    """
    path_length =[]
    path_travel= []
    path_walk = []
    path_wait = [] 
    path_congestion = []
    path_penalty = []
    path_fare = []
    for i in range(len(path_set)):
        link_sque = path_set['link_sque'].iloc[i]
        one_path = link_table[link_table['link_id'].isin(link_sque)]
        path_length.append(one_path['l_a'].sum())
        path_travel.append(one_path['travel'].sum())
        path_walk.append(one_path['walk'].sum())
        path_wait.append(one_path['wait'].sum())
        path_congestion.append(one_path['congestion'].sum())
        path_penalty.append(one_path['penalty'].sum())
        path_fare.append(one_path['fare_rate'].sum())
    
    path_set['path_length'] = path_length
    path_set['path_travel'] = path_travel
    path_set['path_walk'] = path_walk
    path_set['path_wait'] = path_wait
    path_set['path_congestion'] = path_congestion
    path_set['path_penalty'] = path_penalty
    path_set['path_fare'] = path_fare
    
    return path_set

In [19]:
get_path_info(path_set=path_set)

,OD_pair,path_id,mode,link_sque,name_sque,path_length,path_travel,path_walk,path_wait,path_congestion,path_penalty,path_fare
0,r1-s1,path-1,auto,"[43, 5, 2, 1, 3, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",21.0,0,0,0,0,0,0
1,r1-s1,path-2,auto,"[43, 5, 2, 1, 3, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",21.0,0,0,0,0,0,0
2,r1-s1,path-3,auto,"[43, 5, 2, 1, 3, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N6, N6-...",26.0,0,0,0,0,0,0
3,r1-s1,path-4,auto,"[43, 5, 2, 1, 3, 11, 15, 16, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-...",21.0,0,0,0,0,0,0
4,r1-s1,path-5,auto,"[43, 5, 2, 1, 3, 11, 15, 18, 20, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N9, N9-...",26.0,0,0,0,0,0,0
5,r1-s1,path-6,P&R,"[43, 5, 2, 1, 3, 38, 31, 32, 34, 47]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N5, N5-N22, N2...",22.0,0,5,5,0,12,20
7,r1-s1,path-7,auto,"[43, 5, 2, 1, 4, 13, 44]","[r1-N3, N3-N4, N4-N1, N1-N2, N2-N7, N7-N11, N1...",19.0,0,0,0,0,0,0
8,r1-s1,path-8,auto,"[43, 5, 6, 7, 8, 13, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N7, N7-N11, N1...",15.0,0,0,0,0,0,0
9,r1-s1,path-9,auto,"[43, 5, 6, 7, 12, 16, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N11, ...",15.0,0,0,0,0,0,0
10,r1-s1,path-10,auto,"[43, 5, 6, 7, 12, 18, 20, 44]","[r1-N3, N3-N4, N4-N5, N5-N6, N6-N10, N10-N13, ...",20.0,0,0,0,0,0,0
